### **装载云盘**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### **安装tensorflow 2.0**

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1 

In [0]:
# tensorlayer 兼容问题
!pip install imgaug==0.2.6

In [0]:
!pip install tensorlayer

### **cd命令**

In [0]:
import os
os.chdir("/content/drive/My Drive/RL_EA/Deep_Q_Network")
# os.chdir("/usr/local/cuda/include/")

### **查看当前路径**

In [12]:
!pwd
# !cat /usr/local/cuda/version.txt
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
# !nvidia-smi

/content/drive/My Drive/RL_EA/Deep_Q_Network


# Deep Q Network

$$
off-Policy\\
discrete
$$ 

两个特点：

1）从 $Experience\ Replay\ Memory$ 中均匀采样。为了消除样本相关性。

2）延迟复制权值的 $Fixed\ Q-target$ 网络。提高稳定性，收敛性，消除估计$Q$和目标$Q$的相关性。

回顾$Q-Learning$的更新过程:

$$
Q(s,a)=Q(s,a)+\alpha\big[r+\gamma\max_{a'}Q(s',a')-Q(s,a)\big]
$$

可以看到$Q-Learning$就是让$Q$值接近目标$Q$值，那么$DQN$的$loss$：

$$
targetQ\:or\:y'=r + \gamma \max_{a'} Q(s',a';\theta_i^-)
$$
$$
L_i(\theta_i) = \mathbb{E}_{(s,a,r,s') \sim U(D)} \big[ \big( r + \gamma \max_{a'} Q(s',a';\theta_i^-) - Q(s, a; \theta_i) \big)^2 \big]
$$

## DQN的变种：

### Double DQN：
$$
Y_t^{DoubleQ} = R_{t+1} + \gamma  Q(S_{t+1}, \arg\max_a Q(S_{t+1}, a; \theta_t); \theta_t)
$$
减少过估计带来的影响。

### Dueling DQN：
$$
Q(s, a; \theta, \alpha, \beta) = V (s; \theta, \beta) + \big( A(s, a; \theta, \alpha) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s, a'; \theta, \alpha) \big)
$$
同一个网络输出状态值和优势值。

### Prioritized ER: 

是改进效果最好的一个DQN变种。按照 $\delta_i=TD-error$ 的值的大小对ER中的经验进行排序，每个经验的概率为

$$
P(i) = \frac{p_i^{\alpha}}{\sum_k p_k^{\alpha}}
$$

其中 $p_i = |\delta_i| + \epsilon$。考虑到不能只学$P(i)$大的样本，导致过拟合，因此对其进行加权：

$$
w_i = \big( \frac{1}{N} \cdot \frac{1}{P(i)} \big)^\beta
$$

### Noisy DQN：
对网络中的某些（或全部）权值加上noise。也是为了防过拟合，过相关。



In [0]:
# 实现prioritized+DQN
import time
import os
import random
import operator

import numpy as np

import gym
import tensorflow as tf
import tensorlayer as tl
from tutorial_wrappers import build_env

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

ENV_NAME = 'CartPole-v0'  # PongNoFrameskip-v4 or CartPole-v0
env = build_env(ENV_NAME)

In [0]:
### hyper ###

if ENV_NAME == 'CartPole-v0':
  qnet_type = 'MLP'
  number_timesteps = 10000
  explore_timesteps = 100
  # 利用率从0.01 -> 0.99  有些python版本里要改成 1.0*i_iter
  epsilon = lambda i_iter: 1 - 0.99 * min(1, i_iter / explore_timesteps)
  lr = 5e-3
  buffer_size = 1000
  target_q_update_freq = 50
  ob_scale = 1.0
else:
  qnet_type = 'CNN'
  number_timesteps = int(1e6)
  explore_timesteps = 1e5
  # 利用率从0.01 -> 0.99
  epsilon = lambda i_iter: 1 - 0.99 * min(1, i_iter / explore_timesteps)
  lr = 1e-4
  buffer_size = 10000
  target_q_update_freq = 200
  ob_scale = 1.0 / 255

in_dim = env.observation_space.shape
out_dim = env.action_space.n
reward_gamma = 0.99
batch_size = 32
warm_start = buffer_size / 10
prioritized_replay_alpha = 0.6
prioritized_replay_beta0 = 0.4

summary_writer = tf.summary.create_file_writer("logs/")

### **网络框架**

In [0]:
# 动态构建网络方式
class MLP(tl.models.Model):

  def __init__(self, name):
    super(MLP, self).__init__(name=name)
    self.h1 = tl.layers.Dense(64, tf.nn.tanh, in_channels=in_dim[0])
    self.qvalue = tl.layers.Dense(out_dim, in_channels=64, name='q', W_init=tf.initializers.GlorotUniform())
    self.svalue = tl.layers.Dense(1, in_channels=64, name='s', W_init=tf.initializers.GlorotUniform())

  def forward(self, ni):
    feature = self.h1(ni)

    qvalue = self.qvalue(feature)
    svalue = self.svalue(feature)
    # Dueling DQN 公式
    out = svalue + qvalue - tf.reduce_mean(qvalue, 1, keepdims=True)
    return out


class CNN(tl.models.Model):

  def __init__(self, name):
    super(CNN, self).__init__(name=name)
    h, w, in_channels = in_dim
    dense_in_channels = 64 * ((h - 28) // 8) * ((w - 28) // 8)
    self.conv1 = tl.layers.Conv2d(
        32, (8, 8), (4, 4), tf.nn.relu, 'VALID', in_channels=in_channels, name='conv2d_1',
        W_init=tf.initializers.GlorotUniform()
    )
    self.conv2 = tl.layers.Conv2d(
        64, (4, 4), (2, 2), tf.nn.relu, 'VALID', in_channels=32, name='conv2d_2',
        W_init=tf.initializers.GlorotUniform()
    )
    self.conv3 = tl.layers.Conv2d(
        64, (3, 3), (1, 1), tf.nn.relu, 'VALID', in_channels=64, name='conv2d_3',
        W_init=tf.initializers.GlorotUniform()
    )
    # 展开成vector
    self.flatten = tl.layers.Flatten(name='flatten')
    self.preq = tl.layers.Dense(
        256, tf.nn.relu, in_channels=dense_in_channels, name='pre_q', W_init=tf.initializers.GlorotUniform()
    )
    self.qvalue = tl.layers.Dense(out_dim, in_channels=256, name='q', W_init=tf.initializers.GlorotUniform())
    self.pres = tl.layers.Dense(
        256, tf.nn.relu, in_channels=dense_in_channels, name='pre_s', W_init=tf.initializers.GlorotUniform()
    )
    self.svalue = tl.layers.Dense(1, in_channels=256, name='s', W_init=tf.initializers.GlorotUniform())


  def forward(self, ni):
    feature = self.flatten(self.conv3(self.conv2(self.conv1(ni))))
    # preq 和 pres 分别是 V和A的NN层
    qvalue = self.qvalue(self.preq(feature))
    svalue = self.svalue(self.pres(feature))
    # Dueling DQN 公式
    out = svalue + qvalue - tf.reduce_mean(qvalue, 1, keepdims=True)
    return out



### **$Priorizised\ \ ER$ 用到的线段树结构**

In [0]:
class SegmentTree:

  def __init__(self, capacity, operation, neutral_element):
    """
    capacity: array的size，2的幂
    operation: 整合elements的操作
    neutral_element: 对上述操作的边界element
    """
    assert capacity > 0 and capacity & (capacity - 1) == 0,\
    "capacity must be positive and a power of 2"
    self._capacity = capacity
    # 完全二叉树的结构存储数据，叶子节点是数据值（这里是优先权重P）
    self._value = [neutral_element for _ in range(2 * capacity)]
    self._operation = operation

  def _reduce_helper(self, start, end, node, node_start, node_end):
    # logN 操作复杂度
    if start == node_start and end == node_end:
      return self._value[node]
    mid = (node_start + node_end) // 2
    if end <= mid:
      return self._reduce_helper(start, end, 2 * node, node_start, mid)
    else:
      if mid + 1 <= start:
        return self._reduce_helper(start, end, 2 * node + 1, mid + 1, node_end)
      else:
        return self._operation(
            self._reduce_helper(start, mid, 2 * node, node_start, mid),
            self._reduce_helper(mid + 1, end, 2 * node + 1, mid + 1, node_end)
        )
  
  def reduce(self, start=0, end=None):
    """
    返回一个邻近子区间array
    """
    if end is None:
      end = self._capacity
    if end < 0:
      end += self._capacity #???
    end -= 1
    return self._reduce_helper(start, end, 1, 0, self._capacity - 1)

  def __setitem__(self, idx, val):
    # 叶子的index，类似java的get，set，这样做可以直接用数组形式访问。
    idx += self._capacity
    self._value[idx] = val
    idx //= 2
    while idx >= 1:
      self._value[idx] = self._operation(self._value[2 * idx], self._value[2 * idx + 1])
      idx //= 2

  def __getitem__(self, idx):
    assert 0 <= idx < self._capacity
    return self._value[self._capacity + idx]

# Sum 线段树
class SumSegmentTree(SegmentTree):

  def __init__(self, capacity):
    super(SumSegmentTree, self).__init__(capacity=capacity, operation=operator.add, neutral_element=0.0)

  def sum(self, start=0, end=None):
    """
    return: arr[start]+...+arr[end]
    """  
    return super(SumSegmentTree, self).reduce(start, end)

  def find_prefixsum_idx(self, prefixsum):
    """
    找到最大的index i，满足sum(arr[0]+...+arr[i-1]<=prefixsum)

    如果array里是概率，这个函数可以从离散概率中sample index
    
    prefixsum: 之前的elements的和的上界
    """
    assert 0 <= prefixsum <= self.sum() + 1e-5
    idx = 1
    while idx < self._capacity:
      if self._value[2 * idx] > prefixsum:
        idx = 2 * idx
      else:
        prefixsum -= self._value[2 * idx]
        idx = 2 * idx + 1
    return idx - self._capacity #???

# Min线段树
class MinSegmentTree(SegmentTree):

  def __init__(self, capacity):
    # 实际上不用Min树也行，直接对ER池的线段树求min。
    super(MinSegmentTree, self).__init__(capacity=capacity, operation=min, neutral_element=float('inf'))

  def min(self, start=0, end=None):
    """
    返回最小的arr[i]
    """
    return super(MinSegmentTree, self).reduce(start, end)

### **$Experience\ \ ReplayBuffer$**

In [0]:
class ReplayBuffer:

  def __init__(self, size):
    self._storage = []
    self._maxsize = size
    self._next_idx = 0

  def __len__(self):
    return len(self._storage)

  def add(self, *args):
    if self._next_idx >= len(self._storage):
      self._storage.append(args)
    else:
      self._storage[self._next_idx] = args
    self._next_idx = (self._next_idx + 1) % self._maxsize

  def _encode_sample(self, idxes):
    b_o, b_a, b_r, b_o_, b_d = [], [], [], [], []
    for i in idxes:
      o, a, r, o_, d = self._storage[i]
      b_o.append(o)
      b_a.append(a)
      b_r.append(r)
      b_o_.append(o_)
      b_d.append(d)
    return (
        np.stack(b_o).astype('float32') * ob_scale,
        np.stack(b_a).astype('int32'),
        np.stack(b_r).astype('float32'),
        np.stack(b_o_).astype('float32') * ob_scale,
        np.stack(b_d).astype('bool'),
    )

  def sample(self, batch_size):
    indexes = range(len(self._storage))
    idxes = [random.choice(indexes) for _ in range(batch_size)]
    return self._encode_sample(idxes)

# prioritized ER
class PrioritizedReplayBuffer(ReplayBuffer):

  def __init__(self, size, alpha, beta):
    """
    alpha: 0=不优先，1=完全优先
    """
    super(PrioritizedReplayBuffer, self).__init__(size)
    assert alpha >= 0
    self._alpha = alpha
  
    it_capacity = 1
    while it_capacity < size:
      it_capacity *= 2

    self._it_sum = SumSegmentTree(it_capacity)
    self._it_min = MinSegmentTree(it_capacity)
    self._max_priority = 1.0
    self.beta = beta

  def add(self, *args):
    idx = self._next_idx
    super().add(*args)
    # 新加入的数据，给它最大的优先权，防止新数据不会被学习。 
    self._it_sum[idx] = self._max_priority**self._alpha
    self._it_min[idx] = self._max_priority**self._alpha

  def _sample_proportinal(self, batch_size):
    res = []
    p_total = self._it_sum.sum(0, len(self._storage) - 1)
    every_range_len = p_total / batch_size
    for i in range(batch_size):
      mass = random.random() * every_range_len + i * every_range_len
      idx = self._it_sum.find_prefixsum_idx(mass)
      res.append(idx)
    return res

  def sample(self, batch_size):
    idxes = self._sample_proportinal(batch_size)

    it_sum = self._it_sum.sum()
    p_min = self._it_min.min() / it_sum # p_min 是为了计算 w_max
    max_weight = (p_min * len(self._storage))**(-self.beta) # w_max 是为了scale w(i) 见公式

    p_samples = np.asarray([self._it_sum[idx] for idx in idxes]) / it_sum # 求p(i)
    weights = (p_samples * len(self._storage))**(-self.beta) / max_weight # 求w(i)
    encoded_sample = self._encode_sample(idxes)
    return encoded_sample + (weights, idxes)

  def update_priorities(self, idxes, priorities):
    assert len(idxes) == len(priorities)
    for idx, priority in zip(idxes, priorities):
      assert priority > 0
      assert 0 <= idx < len(self._storage)
      self._it_sum[idx] = priority**self._alpha
      self._it_min[idx] = priority**self._alpha

      self._max_priority = max(self._max_priority, priority)

### **$Utils$**

In [0]:
def huber_loss(x):
  return tf.where(tf.abs(x) < 1, tf.square(x) * 0.5, tf.abs(x) - 0.5)

def sync(net, net_tar):
  for var, var_tar in zip(net.trainable_weights, net_tar.trainable_weights):
    var_tar.assign(var)

### **训练**

In [19]:
# train
qnet = MLP('q') if qnet_type == 'MLP' else CNN('q')
qnet.train()
trainable_weights = qnet.trainable_weights
targetqnet = MLP('targetq') if qnet_type == 'MLP' else CNN('targetq')
targetqnet.infer()
sync(qnet, targetqnet)
optimizer = tf.optimizers.Adam(learning_rate=lr)

'''
这个buffer的线段树是有2n个点的，结构是完全二叉树，前n-1个点是后n个点的sum，第0个点不用，实际访问数是访问后n个数，
寻找数的时候从1开始，依次*2 或*2+1 访问子节点。
'''
buffer = PrioritizedReplayBuffer(buffer_size, prioritized_replay_alpha, prioritized_replay_beta0)

o = env.reset()
nepisode = 0
t = time.time()
for i in range(1, number_timesteps + 1):
  eps = epsilon(i)
  buffer.beta += (1 - prioritized_replay_beta0) / number_timesteps
  
  # 选择动作
  if random.random() < eps:
    a = int(random.random() * out_dim)
  else:
    obv = np.expand_dims(o, 0).astype('float32') * ob_scale
    a = qnet(obv).numpy().argmax(1)[0]

  # 执行动作，保存经验数据
  o_, r, done, info = env.step(a)
  buffer.add(o, a, r, o_, done)

  if i >= warm_start:
    # 延迟更新target
    if i % target_q_update_freq == 0:
      sync(qnet, targetqnet)
      
      if i % 5000 == 0:
        path = os.path.join('model/', '{}_{}.npz'.format(qnet_type, i))
        tl.files.save_npz(qnet.trainable_weights, name=path)

    # sample数据
    b_o, b_a, b_r, b_o_, b_d, weights, idxs = buffer.sample(batch_size)

    #q' 估计
    b_q_ = (1 - b_d) * tf.reduce_max(targetqnet(b_o_), 1)

    # loss
    with tf.GradientTape() as q_tape:
      b_q = tf.reduce_sum(qnet(b_o) * tf.one_hot(b_a, out_dim), 1)
      abs_td_error = tf.abs(b_q - (b_r + reward_gamma * b_q_))
      priorities = np.clip(abs_td_error.numpy(), 1e-6, None)
      buffer.update_priorities(idxs, priorities)
      loss = tf.reduce_mean(weights * huber_loss(abs_td_error))

    q_grad = q_tape.gradient(loss, trainable_weights)
    optimizer.apply_gradients(zip(q_grad, trainable_weights))

  if done:
    o = env.reset()
  else:
    o = o_

  # episode in info is real (unwrapped) message ???
  if info.get('episode'):
    nepisode += 1
    reward, length = info['episode']['r'], info['episode']['l']
    fps = int(length / (time.time() - t))
    print(
        'Time steps fo far:{}, episode so fa:{}, episode reward:{:.4f}, episode length:{}, FPS:{}'\
        .format(i, nepisode, reward, length, fps)
    )
    t = time.time()
    with summary_writer.as_default():
      tf.summary.scalar('reward', reward, step=nepisode)


Time steps fo far:21, episode so fa:1, episode reward:21.0000, episode length:21, FPS:125
Time steps fo far:39, episode so fa:2, episode reward:18.0000, episode length:18, FPS:3612
Time steps fo far:49, episode so fa:3, episode reward:10.0000, episode length:10, FPS:1116
Time steps fo far:60, episode so fa:4, episode reward:11.0000, episode length:11, FPS:1062
Time steps fo far:69, episode so fa:5, episode reward:9.0000, episode length:9, FPS:1310
Time steps fo far:81, episode so fa:6, episode reward:12.0000, episode length:12, FPS:1161
Time steps fo far:90, episode so fa:7, episode reward:9.0000, episode length:9, FPS:1556
Time steps fo far:100, episode so fa:8, episode reward:10.0000, episode length:10, FPS:140
Time steps fo far:109, episode so fa:9, episode reward:9.0000, episode length:9, FPS:49
Time steps fo far:118, episode so fa:10, episode reward:9.0000, episode length:9, FPS:51
Time steps fo far:126, episode so fa:11, episode reward:8.0000, episode length:8, FPS:50
Time steps 

### **测试**

In [20]:
# test
qnet = MLP('q_t0') if qnet_type == 'MLP' else CNN('q_t0')
# tl.files.load_and_assign_npz(name='model/{}.npz'.format(20800))
tl.files.load_and_assign_npz(name='model/CartPole_10000.npz', network=qnet)
qnet.eval()

nepisode = 0
o = env.reset()
for i in range(1, number_timesteps + 1):
  # env.render()
  obv = np.expand_dims(o, 0).astype('float32') * ob_scale
  a = qnet(obv).numpy().argmax(1)[0]
   
  o_, r, done, info = env.step(a)

  if done:
    o = env.reset()
  else:
    o = o_
  
  # ???
  if info.get('episode'):
    nepisode += 1
    reward, length = info['episode']['r'], info['episode']['l']
    print(
        'Time steps so fa:{}, episode so far:{}, episode reward:{:.4f}, episode length:{}'\
        .format(i, nepisode, reward, length)
    )

Time steps so fa:200, episode so far:1, episode reward:200.0000, episode length:200
Time steps so fa:400, episode so far:2, episode reward:200.0000, episode length:200
Time steps so fa:600, episode so far:3, episode reward:200.0000, episode length:200
Time steps so fa:800, episode so far:4, episode reward:200.0000, episode length:200
Time steps so fa:1000, episode so far:5, episode reward:200.0000, episode length:200
Time steps so fa:1200, episode so far:6, episode reward:200.0000, episode length:200
Time steps so fa:1400, episode so far:7, episode reward:200.0000, episode length:200
Time steps so fa:1600, episode so far:8, episode reward:200.0000, episode length:200
Time steps so fa:1800, episode so far:9, episode reward:200.0000, episode length:200
Time steps so fa:2000, episode so far:10, episode reward:200.0000, episode length:200
Time steps so fa:2200, episode so far:11, episode reward:200.0000, episode length:200
Time steps so fa:2400, episode so far:12, episode reward:200.0000, 